Downloads description of articles
Cargo un archivo en forma de dataframe, exitraigo un sub-dataframe, scrapeo el comentario de wallapop y lo retorno en un df que es id - descripcion

In [2]:
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [38]:
elemento_a_buscar = "iphone 15" # para guardar el archivo
iphone = 15
estado = 'as_good_as_new'
carpeta = "3_datos_finales"
fecha = "20241114"

In [ ]:
# Cargo archivo

df_original = pd.read_excel(carpeta + '/' + fecha + '.xlsx')
df_original.head()


,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1068541305,20241114,16,basic,128,NaN,"800,00",new,iphone 16 128gb,False,https://es.wallapop.com/item/iphone-16-128gb-1...,False,False,False
1,1069277594,20241114,14,pro,NaN,NaN,"700,00",new,iphone 14 pro morado,False,https://es.wallapop.com/item/iphone-14-pro-mor...,True,False,False
2,1069385453,20241114,16,basic,NaN,NaN,"850,00",as_good_as_new,iphone 16,False,https://es.wallapop.com/item/iphone-16-1069385453,False,False,False
3,1069408278,20241114,14,pro,NaN,NaN,"800,00",good,iphone 14 pro,False,https://es.wallapop.com/item/iphone-14-pro-106...,False,False,False
4,1069426273,20241114,16,pro max,NaN,NaN,"1020,00",good,iphone 16 pro max,False,https://es.wallapop.com/item/iphone-16-pro-max...,False,False,False


In [ ]:
# Selecciono el sub-dataframe.
# Selecciono:       as_good_as_new 
#                   iphone 15. 
#                   bateria = NaN
# Son 263 apariciones
df_to_scrap = df_original[(df_original['estado'] == estado) & 
                          (df_original['gen'] == iphone) & 
                          (pd.isna(df_original['bateria']))]
df_to_scrap.reset_index(drop=True, inplace = True)
df_to_scrap.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1069439182,20241114,15,basic,128,NaN,"619,00",as_good_as_new,iphone 15 128gb como nuevo!,False,https://es.wallapop.com/item/iphone-15-128gb-c...,False,True,False
1,1069438545,20241114,15,pro max,256,NaN,"879,00",as_good_as_new,iphone 15 pro max 256gb como nuevo!,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,True,False
2,1069441441,20241114,15,pro max,256,NaN,"910,00",as_good_as_new,iphone 15 pro max esim 256gb,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False
3,1069386886,20241114,15,pro max,NaN,NaN,"850,00",as_good_as_new,iphone 15 pro max,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False
4,1069216370,20241114,15,basic,NaN,NaN,"700,00",as_good_as_new,iphone 15,False,https://es.wallapop.com/item/iphone-15-1069216370,False,False,False


In [5]:
# Función de scrapeo comentarios
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# no existe https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182
# si existe https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441

In [13]:
import time
# 263 anuncios - 25 min. con sleep de 1
# Escrapeo objtos de:
# Vendedor, cuadro datos, cuadro envio, cuadro comentario, cuadro 'editado views y likes'
# por ahora solo comentario
def scrap_comments(df):
    df_len = len(df)
    df['comentario'] = None
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(df.loc[0]['url']) 
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click() # quita cookies y carga primera web
    for index in range(len(df)):
        driver.get(df.loc[index]['url']) 
        print(index, '/', df_len)
        try:                                                                                    # si existe la web
            elemento_comentario = driver.find_element(By.CSS_SELECTOR, "#__next > main > div > div.item-detail_ItemDetail__container__8p25r.pb-5 > section.justify-content-around.d-flex.py-4 > div.item-detail_ItemDetail__card__jnUEv.pb-5 > div.px-4 > div.item-detail_ItemDetail__separator__SCH3p.py-2 > section").text
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url'], ': \n', df.loc[index]['comentario'])                                                                                     
        except:                                                                              # Si no existe la web: 
            element = driver.find_element(By.CLASS_NAME, "not-found-page_Error__title__Dky_8")
            elemento_comentario = "not found"
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url'], df.loc[index, 'comentario'])
        #time.sleep(1)
    driver.quit()
    return (df)

df_with_comments = scrap_comments(df_to_scrap)

C:\Users\ACER\AppData\Local\Temp\ipykernel_19556\4260173602.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comentario'] = None


0 / 263
https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182 not found
1 / 263
https://es.wallapop.com/item/iphone-15-pro-max-256gb-como-nuevo-1069438545 not found
2 / 263
https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441 : 
 BATERIA 100% ✅
PRIMER USO EN JUNIO ✅
Tecnologia eSIM olvidate de la sim tradicional ✅
FACTURA ✅
CABLE DE CARGA ✅
GARANTIA APPLE ✅
COMO NUEVO ✅
ENTREGA EN MANO EN MADRID O ALREDEDORES ✅
3 / 263
https://es.wallapop.com/item/iphone-15-pro-max-1069386886 : 
 VENDO Iphone 15 Pro Max en muy buen estado con un uso de de 2 meses solo esta dañado en la esquina trasera pero en general todo va fenomenal, tiene un 100 de bateria todo va perfectamente lo vendo porque tengo el 16 
viene en su caja original
4 / 263
https://es.wallapop.com/item/iphone-15-1069216370 : 
 iPhone 15 128GB como nuevo. Siempre ha estado con cristal templado y con protector de cámara. Lo entrego en su caja y el cargador originales. Regalo funda original de Apple y varios

In [11]:
df_with_comments.head(4)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1069439182,20241114,15,basic,128,NaN,"619,00",as_good_as_new,iphone 15 128gb como nuevo!,False,https://es.wallapop.com/item/iphone-15-128gb-c...,False,True,False,None
1,1069438545,20241114,15,pro max,256,NaN,"879,00",as_good_as_new,iphone 15 pro max 256gb como nuevo!,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,True,False,None
2,1069441441,20241114,15,pro max,256,NaN,"910,00",as_good_as_new,iphone 15 pro max esim 256gb,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False,BATERIA 100% ✅\nPRIMER USO EN JUNIO ✅\nTecnolo...
3,1069386886,20241114,15,pro max,NaN,NaN,"850,00",as_good_as_new,iphone 15 pro max,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False,VENDO Iphone 15 Pro Max en muy buen estado con...


In [17]:
df_original = df_original.drop('comments', axis=1)
df_original = df_original.drop('comentario_x', axis=1)
df_original = df_original.drop('comentario_y', axis=1)

In [27]:
subdf_comentarios = df_with_comments[['id', 'comentario']]
subdf_comentarios

,id,comentario
0,1069439182,not found
1,1069438545,not found
2,1069441441,BATERIA 100% ✅\nPRIMER USO EN JUNIO ✅\nTecnolo...
3,1069386886,VENDO Iphone 15 Pro Max en muy buen estado con...
4,1069216370,iPhone 15 128GB como nuevo. Siempre ha estado ...
...,...,...
258,1064592606,"Iphone 15 Pro titiano natural, como nuevo, muy..."
259,1065025766,Vendo Iphone 15 Pro Max en perfecto estado. 99...
260,1065271790,not found
261,1058842793,"iPhone 15 Pro usado en buen estado!, como nuev..."


In [28]:
# Guardo comentarios finales en el df original
df_original = pd.merge(df_original, subdf_comentarios, on='id', how='left')


In [32]:
df_original.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1068541305,20241114,16,basic,128,NaN,"800,00",new,iphone 16 128gb,False,https://es.wallapop.com/item/iphone-16-128gb-1...,False,False,False,NaN
1,1069277594,20241114,14,pro,NaN,NaN,"700,00",new,iphone 14 pro morado,False,https://es.wallapop.com/item/iphone-14-pro-mor...,True,False,False,NaN
2,1069385453,20241114,16,basic,NaN,NaN,"850,00",as_good_as_new,iphone 16,False,https://es.wallapop.com/item/iphone-16-1069385453,False,False,False,NaN
3,1069408278,20241114,14,pro,NaN,NaN,"800,00",good,iphone 14 pro,False,https://es.wallapop.com/item/iphone-14-pro-106...,False,False,False,NaN
4,1069426273,20241114,16,pro max,NaN,NaN,"1020,00",good,iphone 16 pro max,False,https://es.wallapop.com/item/iphone-16-pro-max...,False,False,False,NaN


In [39]:
nombre_archivo_csv = df_original
carpeta = "4_download_description"
nombre_archivo_pkl = fecha + '_' + elemento_a_buscar + '_' + estado
df_original.to_csv(carpeta + '/' + nombre_archivo_pkl + '.csv')